In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["HF_HOME"]="/export/a02/huggingface"

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import os
import torch

In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [16]:
# model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ", torch_dtype=torch.float16, device_map="auto", quantization_config=gptq_config)
# model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-GPTQ", device_map="auto")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6b", torch_dtype=torch.float16, device_map="auto")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype=torch.float16, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ")
print([n for n, p in model.named_parameters()])
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.2.self_attn.k_proj.weight', 'model.layers.2.self_attn.v_proj.weight', 'model.layers.2.self_attn.o_proj.weight', 'model.layers.2.mlp.gate_proj.weight', 'model.layers.2

In [17]:
print(type(model.model.layers[29].self_attn))

<class 'transformers.models.llama.modeling_llama.LlamaAttention'>


In [14]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [4]:
model.config.eos_token_id

2

In [5]:
generation_config = GenerationConfig(
    max_length=1024,
    max_new_tokens=512,
    temperature=1.0,
    do_sample=True,
    num_return_sequences=1,
    use_cache=True
)
# generation_config = GenerationConfig(
#     max_length=1024,
#     max_new_tokens=512,
#     temperature=1.0,
#     do_sample=True,
#     num_return_sequences=1,
#     use_cache=True,
#     pad_token_id=tokenizer.unk_token_id
# )

In [6]:
inputs = tokenizer("Once upon a time, 200 plus years ago, to be precise, when sailing ships were still roaming the seven seas and the British navy ruled the waves on which the pirates and privateers alike sailed, in the tiny island of Montserrat was born the daughter of a well-respected merchant.", return_tensors="pt").to("cuda")

In [7]:
%%time 
outputs = model.generate(**inputs, generation_config=generation_config)

CPU times: user 30.5 s, sys: 175 ms, total: 30.7 s
Wall time: 30.7 s


In [8]:
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

Once upon a time, 200 plus years ago, to be precise, when sailing ships were still roaming the seven seas and the British navy ruled the waves on which the pirates and privateers alike sailed, in the tiny island of Montserrat was born the daughter of a well-respected merchant.
Her mother raised her as a dutiful housewife. She attended the village church often and taught the young men as a Sunday school teacher.
As she approached the age of maturing, she had grown into a gorgeous and delicate young woman.
Her father sent her off to England to learn how to run a proper household. She had little interest; this daughter of a merchant was born to be married into the gentry class, thus she was destined to be a housewife, one of many who were to be her peers. And she was also to have heirs, sons to carry on her husband’s name and fortune.
She married a handsome lad of the gentiles a year after her arrival in England, and her life then took a bit of a twisted turn. Shortly after marrying him, 

In [11]:
outputs.numel()

583

In [12]:
inputs["input_ids"].numel()

71